In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials #To access authorised Spotify data
import spotipy.util as util
import json
import pprint

client_id = ''
client_secret = ''
username = ''
scope = 'user-library-read'
redirect_uri = 'http://localhost:7778/callback'
token_obj = util.prompt_for_user_token(
    username=username, 
    scope=scope, 
    client_id=client_id,   
    client_secret=client_secret,
    redirect_uri=redirect_uri
)

In [8]:
def get_user_library(token):
    user_saved_tracks = []
    get_next_track_set = True
    offset = 0
    headers = {
        'Authorization': 'Bearer {}'.format(token)
    }
    while get_next_track_set == True:
        params = {'limit' : 50, 'offset' : offset}
        response = requests.get('https://api.spotify.com/v1/me/tracks', headers=headers, params=params)
        tracks_in_set = json.loads(response.text)
        tracks_in_set = tracks_in_set['items']
        if len(tracks_in_set) > 0:
            user_saved_tracks = user_saved_tracks + tracks_in_set
            offset = offset + 50
        elif len(tracks_in_set) == 0:
            get_next_track_set = False
            
    saved_tracks_formatted = []
    for obj in user_saved_tracks:
        added_at = obj['added_at']
        track = obj['track']
        
        track_formatted = {
            'track_id' : track['id'],
            'saved_at' : added_at,
            'track_name' : track['name'],
            'primary_artist_id' : track['artists'][0]['id'],
            'all_artists' : [artist['id'] for artist in track['artists']],
            'album_id' : track['album']['id'],
            'duration_ms' : track['duration_ms'],
            'is_explicit' : track['explicit'],
            'popularity': track['popularity'],
            'track_number' : track['track_number']
        }
        saved_tracks_formatted.append(track_formatted)
        
    saved_tracks_df = pd.DataFrame(saved_tracks_formatted)
    list_of_track_ids = list(saved_tracks_df['track_id'])
    
    all_track_features = []
    lower_bound = 0
    upper_bound = 99
    for interval in range(0,len(list_of_track_ids),100):
        tracks_to_get_features = list_of_track_ids[lower_bound:upper_bound]
        tracks_string = ','.join(tracks_to_get_features)
        track_features = requests.get('https://api.spotify.com/v1/audio-features/', headers=headers, params={'ids': tracks_string})
        status_code = track_features.status_code
        while status_code == 503:
            track_features = requests.get('https://api.spotify.com/v1/audio-features/', headers=headers, params={'ids': tracks_string})
            status_code = track_features.status_code
        track_features = json.loads(track_features.text)['audio_features']
        all_track_features = all_track_features + track_features
        lower_bound = upper_bound
        upper_bound += 100
    
    track_features_df = pd.DataFrame(all_track_features)
    
    saved_tracks_df = saved_tracks_df.merge(track_features_df, how='left', left_on='track_id', right_on='id')
    
    saved_tracks_df = saved_tracks_df[
        [
            'album_id', 'all_artists', 'duration_ms_x', 'is_explicit', 'popularity',
            'primary_artist_id', 'track_id', 'track_name', 'track_number',
            'acousticness', 'danceability','energy', 'instrumentalness', 
            'key', 'liveness', 'loudness','mode', 'speechiness', 'tempo', 
            'time_signature', 'valence','saved_at'
        ]
    ]
    
    saved_tracks_df = saved_tracks_df.rename(columns={'duration_ms_x' : 'duration_ms'})
    
    saved_tracks_df = saved_tracks_df[['track_id','saved_at','track_name','primary_artist_id','all_artists','album_id','track_number','duration_ms','popularity','is_explicit','acousticness','danceability','energy','instrumentalness','key','liveness','loudness','mode','speechiness','tempo','time_signature','valence']]
    
    return saved_tracks_df

In [9]:
library_df = get_user_library(token_obj)

library_df

,track_id,saved_at,track_name,primary_artist_id,all_artists,album_id,track_number,duration_ms,popularity,is_explicit,...,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,69E50pIDeFjLYy3uiYC0Zd,2021-03-10T17:41:44Z,Stories,3kANxNTLNOhxpOPoCbGq9E,[3kANxNTLNOhxpOPoCbGq9E],1dozAAM6CMyL2FAEUcHJKu,3,135534,42,False,...,0.499,0.000000,1,0.1970,-9.716,1,0.0645,87.673,4,0.2440
1,3FLazwxIsNIWPmND1t1RIx,2021-03-10T16:04:10Z,Oh No,3uWJaTLnUnp0wZfB5xcdJy,[3uWJaTLnUnp0wZfB5xcdJy],1cvUueJ1VO2gy8HaOKytad,1,285954,43,False,...,0.665,0.000056,5,0.0688,-5.799,1,0.0350,109.973,4,0.8530
2,54cUU4IYhhjNzGylSaqGIk,2021-03-06T21:47:05Z,Change Your Mind,3HdQTgQSncptIPjDgskWbu,[3HdQTgQSncptIPjDgskWbu],5CHPfB4oMbUaY06b0jGMGi,3,156920,45,False,...,0.521,0.000171,7,0.1180,-6.417,1,0.0339,107.942,4,0.5880
3,03mmfpJC0IKUUR2FAs8QVj,2021-03-06T21:43:41Z,Not the One,6mcEXkX1gFc4kttIF6JKb5,[6mcEXkX1gFc4kttIF6JKb5],43qi4foeH5DtclYfzPjUdH,1,219789,41,False,...,0.350,0.086800,5,0.1090,-8.269,1,0.0489,94.984,4,0.0908
4,5tVbKij7vNLI0psFkkzyew,2021-03-04T14:10:21Z,Gasoline,4Ui2kfOqGujY81UcPrb5KE,[4Ui2kfOqGujY81UcPrb5KE],6NtEjhPWfZcvJQuvjGX4bk,5,193373,49,False,...,0.714,0.004550,1,0.1170,-4.557,0,0.0612,133.127,4,0.5610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893,601sKQRE4ZVptFWVwLw8k7,2015-04-10T15:52:51Z,Werewolf Gimmick,3hyGGjxu73JuzBa757H6R5,[3hyGGjxu73JuzBa757H6R5],7HWC61Sl93gYdBqCV5EIka,9,154640,35,False,...,0.833,0.020600,9,0.0852,-5.040,1,0.1190,84.087,4,0.6280
4894,2q2HJiWcwLi5xx0SRiQSzt,2015-04-10T15:52:51Z,Luna,3hyGGjxu73JuzBa757H6R5,[3hyGGjxu73JuzBa757H6R5],7HWC61Sl93gYdBqCV5EIka,10,207640,25,False,...,0.405,0.540000,9,0.1230,-14.098,0,0.0324,103.206,4,0.2900
4895,5q8TnYw6NloSe8f1kAX1ph,2015-04-10T15:52:51Z,Unmasked!,3hyGGjxu73JuzBa757H6R5,[3hyGGjxu73JuzBa757H6R5],7HWC61Sl93gYdBqCV5EIka,11,208666,25,False,...,0.198,0.034500,9,0.2060,-14.951,1,0.0290,96.005,4,0.2560
4896,3Emwadz87H7lv9vetO2m0X,2015-04-10T15:52:51Z,The Ballad of Bull Ramos,3hyGGjxu73JuzBa757H6R5,[3hyGGjxu73JuzBa757H6R5],7HWC61Sl93gYdBqCV5EIka,12,173040,26,False,...,0.793,0.013900,1,0.3630,-6.823,0,0.0356,112.607,4,0.8140
